<a href="https://colab.research.google.com/github/smit585/SelfDrivingCar/blob/master/Course%202%20State%20Estimation%20and%20Localization/Week%202%20Karman%20Filters/Extended%20Karman%20Filters%20Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Can look at these links to understand Karman Filter;
* [Karman Filter Working in Pictures](https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/)
* [Medium](https://medium.com/@dingyan7361/least-squares-recursive-least-squares-kalman-filters-and-sensor-fusion-ed13f6242e9e)

# Problem in Extended Karman Filter

![Problem Statement](https://miro.medium.com/max/1400/1*1zaDx4cF-Dph1sNmul8HxQ.png)

We have the formentioned problem statement. Instead of a linear model like GPS, this time we have a camera on the car, and using the Angle of elevation we can measure the position of the car as we know the Height and distance of the statue. As the measurement model is non-linear, therefore we cannot use Linear Karman Filter. Instead, we would use Extended Karman filter, that linearizes the model at an operating point by expanding the model using Taylor Expansion and taking first order terms.

Apart from this model, we have the data as:

![Data](https://miro.medium.com/max/574/1*Uhk1x4J4UhxSL3oL_xDCmg.png)

Now, we need to find the next position and velocity of the vehicle by fusing predicted and measured values.

From all the data present above, we can calculate the Jacobian matrices and State matrices like:

*   Fk = [1 0.5; 0 1]
*   Pk = [0.01 0; 0 1]
*   Qk = [0.1 0; 0 0.1]
* Rk = 0.01

Jacobian Matrices: (As all the models have linear relationship with Errors)
* Lk = 1
* Mk = 1
* Hk = [S/((D-x)**2 + S ** 2), 0]
  * Hk = [20/(1806.25), 0] = [0.011, 0]

In [77]:
import numpy as np
from numpy.linalg import inv

# Loading the initial Data
x0 = np.array([[0],[5]])
delT = 0.5
u = -2
S = 20
D = 40
y0 = np.pi/6

# Creating the matrices
F = np.array([[1, delT], [0, 1]])
P = np.array([[0.01, 0], [0, 1]]) 
Q = np.array([[0.1, 0], [0, 0.1 ]])
G = np.array([[0],[delT]])
R = 0.01
L = 1
M = 1


P_k = F.dot(P.dot(F.T)) + Q
print(P_k)

[[0.36 0.5 ]
 [0.5  1.1 ]]


In [78]:
# Calculating the Predicted State using Motion Model
x = F.dot(x0) + G*u
print(x.item(0))

2.5


In [79]:
# Calculate H matrix using the predicted position
# Predicted position is in zeroth index of the state matrix
H_k = np.array([[S/((D - x.item(0)) **2  + S ** 2), 0]])
print(H_k)

[[0.01107266 0.        ]]


In [80]:
# Find the optimal gain to fuse the predicted and measured data
K_k = P_k.dot(H_k.T.dot(inv(H_k.dot(P_k.dot(H_k.T)) + M*R*M)))
print(K_k)

[[0.39686426]
 [0.55120036]]


In [81]:
# Finally obtain the value of estimated states by fusing predicted and measured
# data and adding it to the innovation. Value of y at initial time step is 
# already present.
x1 = x + K_k.dot(y0 - np.arctan(S/(D - x.item(0))))
print(x1)

[[2.51335109]
 [4.01854318]]
